# Smart FAQ Chatbot with vectorizer

## Step-1: Import Necessary Libraries

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

## Step-2 : FAQChatbot Class Creation

In [2]:
class FAQChatbot:
    def __init__(self, faq_data, similarity_threshold=0.3):
        """
        Initialize the FAQ chatbot with a dictionary of questions and answers.

        Args:
            faq_data (dict): Dictionary with FAQ questions as keys and answers as values
            similarity_threshold (float): Minimum similarity score to consider a match
        """
        self.faq_data = faq_data
        self.questions = list(faq_data.keys())
        self.answers = list(faq_data.values())
        self.threshold = similarity_threshold

        # Initialize and fit the TF-IDF vectorizer on the FAQ questions
        self.vectorizer = TfidfVectorizer(
            lowercase=True,
            strip_accents='unicode',
            ngram_range=(1, 3),  # Include unigrams, bigrams, and trigrams for better matching
            max_features=10000,   # Increased for larger FAQ dataset
            stop_words='english'
        )

        # Create TF-IDF matrix for FAQ questions
        self.question_vectors = self.vectorizer.fit_transform(self.questions)

    def preprocess_query(self, query):
        """
        Preprocess the user query: lowercase, remove punctuation, etc.

        Args:
            query (str): User's question

        Returns:
            str: Preprocessed query
        """
        # Convert to lowercase
        query = query.lower()

        # Remove punctuation except for question marks
        query = re.sub(r'[^\w\s\?]', '', query)

        # Remove extra whitespace
        query = re.sub(r'\s+', ' ', query).strip()

        return query

    def get_answer(self, user_query):
        """
        Find the most similar FAQ question and return its answer.

        Args:
            user_query (str): User's question

        Returns:
            str: Answer to the most similar FAQ question or a default message
        """
        # Preprocess the query
        processed_query = self.preprocess_query(user_query)

        # Transform the user query using the trained vectorizer
        query_vector = self.vectorizer.transform([processed_query])

        # Calculate cosine similarity between user query and all FAQ questions
        similarities = cosine_similarity(query_vector, self.question_vectors)[0]

        # Find the top 3 most similar questions (for potential future use)
        top_indices = np.argsort(similarities)[::-1][:3]
        top_scores = similarities[top_indices]

        # Find the most similar question
        max_similarity_idx = top_indices[0]
        max_similarity_score = top_scores[0]

        # For debugging purposes, you can uncomment the following:
        # print(f"Query: {processed_query}")
        # print(f"Best match: {self.questions[max_similarity_idx]} (Score: {max_similarity_score:.4f})")
        # print(f"2nd match: {self.questions[top_indices[1]]} (Score: {top_scores[1]:.4f})")
        # print(f"3rd match: {self.questions[top_indices[2]]} (Score: {top_scores[2]:.4f})")

        # If the highest similarity is above our threshold, return the corresponding answer
        if max_similarity_score >= self.threshold:
            return self.answers[max_similarity_idx]
        else:
            return "Sorry, I couldn't find a suitable answer to your question."


In [3]:
def get_faq_answer(user_query):
    """
    Function to get answers for user queries from the FAQ data.

    Args:
        user_query (str): User's question

    Returns:
        str: Answer to the user's question or a default message
    """
    return chatbot.get_answer(user_query)

## Step-3: Create a comprehensive FAQ database for an e-commerce store

In [4]:
# Create a comprehensive FAQ database for an e-commerce store
faq_data = {
    # Delivery & Shipping
    "Do you offer delivery?": "Yes, we provide delivery service for all orders above 500 BDT.",
    "What are your delivery options?": "We offer standard delivery (3-5 days), express delivery (1-2 days), and same-day delivery for orders placed before noon.",
    "How much does shipping cost?": "Shipping is free for orders above 1000 BDT. For orders below that, a flat fee of 100 BDT applies.",
    "Do you deliver on weekends?": "Yes, we offer weekend delivery for all areas within Dhaka city. For other locations, weekend delivery may not be available.",
    "How long will it take to receive my order?": "Standard delivery takes 3-5 business days, express delivery takes 1-2 business days, and same-day delivery is available for orders placed before noon in select areas.",
    "Do you offer international shipping?": "Currently, we only ship within Bangladesh.",
    "Which areas do you deliver to?": "We deliver to all major cities and towns in Bangladesh. Remote areas may have limited delivery options.",
    "Can I change my delivery address after placing an order?": "You can change your delivery address within 1 hour of placing your order by contacting our customer service team.",

    # Store Information
    "Where are you located?": "We are located at 123 Main Road, Dhaka.",
    "What are your store hours?": "Our physical store is open from 10 AM to 8 PM, seven days a week.",
    "Do you have a physical store?": "Yes, we have a flagship store at 123 Main Road, Dhaka, and several branches throughout the city.",
    "Do you have multiple branches?": "Yes, we have 5 branches across Dhaka and 2 branches in Chittagong. Visit our website's 'Store Locator' for the full list.",
    "Is parking available at your store?": "Yes, our flagship store offers free parking for customers for up to 2 hours.",

    # Returns & Refunds
    "Can I return a product?": "You can return a product within 7 days of delivery.",
    "What is your return policy?": "We accept returns within 7 days of delivery. The product must be in its original packaging and undamaged.",
    "How do I initiate a return?": "To initiate a return, log into your account, go to 'My Orders', select the order and click on 'Return Item'. Alternatively, you can contact our customer service.",
    "Do you offer refunds?": "Yes, we offer full refunds for returned products that meet our return policy criteria. Refunds typically process within 5-7 business days.",
    "Will I get a refund if my product is defective?": "Yes, we offer immediate replacements or full refunds for defective products. Please contact customer service within 24 hours of receiving the product.",
    "Do you charge a restocking fee for returns?": "No, we don't charge any restocking fees for returns that comply with our return policy.",
    "Can I exchange an item instead of returning it?": "Yes, you can exchange items within 10 days of delivery. The new item must be of equal or greater value.",

    # Order Tracking & Management
    "How do I track my order?": "You can track your order by logging into your account or using the tracking number provided in your order confirmation email.",
    "How do I cancel my order?": "You can cancel your order within 2 hours of placing it by contacting our customer service at support@example.com.",
    "Can I modify my order after placing it?": "Orders can be modified within 1 hour of placement. Please contact customer service immediately with your order number.",
    "What if my order is delayed?": "If your order is delayed beyond the estimated delivery time, please contact our customer service team. We will investigate the issue and provide compensation if applicable.",
    "How will I know when my order has shipped?": "You will receive an email and SMS notification when your order has been shipped with tracking information.",

    # Payment & Pricing
    "What payment methods do you accept?": "We accept credit/debit cards, mobile banking (bKash, Nagad), and cash on delivery.",
    "Is cash on delivery available?": "Yes, cash on delivery is available for all areas within Dhaka city.",
    "Are there any discounts for bulk orders?": "Yes, orders of 10 or more items qualify for a 15% discount.",
    "Do you offer installment payment options?": "Yes, we offer 0% interest installment plans on purchases above 5000 BDT with select bank credit cards.",
    "Are prices inclusive of taxes?": "Yes, all prices listed on our website include VAT and all applicable taxes.",
    "Do you price match?": "Yes, we offer price matching for identical products sold by authorized retailers in Bangladesh. Please contact customer service with proof of the lower price.",
    "How secure is my payment information?": "We use industry-standard SSL encryption to protect your payment information. We never store your full credit card details on our servers.",

    # Customer Service
    "How can I contact customer service?": "You can reach our customer service team at support@example.com or call us at +880-123-456789.",
    "What are your customer service hours?": "Our customer service team is available 7 days a week from 9 AM to 10 PM.",
    "Do you have a live chat option?": "Yes, our live chat support is available on our website from 9 AM to 10 PM every day.",
    "How do I file a complaint?": "You can file a complaint by emailing complaints@example.com or by calling our dedicated complaint line at +880-123-456790.",

    # Account Management
    "How do I create an account?": "You can create an account by clicking on the 'Sign Up' button on our website or during the checkout process.",
    "How do I reset my password?": "Click on the 'Forgot Password' link on the login page and follow the instructions sent to your registered email address.",
    "How do I update my account information?": "Log into your account, go to 'My Profile', and click on 'Edit Profile' to update your information.",
    "Is my personal information secure?": "Yes, we employ strict security measures to protect your personal information and never share it with third parties without your explicit consent.",

    # Products & Services
    "What brands do you carry?": "We carry a wide range of local and international brands including Nike, Adidas, Uniqlo, and many more.",
    "Do you sell authentic products?": "Yes, we only sell 100% authentic products sourced directly from authorized distributors or manufacturers.",
    "What is your warranty policy?": "All electronic items come with a standard 1-year manufacturer warranty unless otherwise specified.",
    "Do you offer product installation services?": "Yes, we offer installation services for large appliances and electronics for an additional fee starting at 300 BDT.",
    "How can I check if a product is in stock?": "The availability of products is shown on each product page. If it says 'In Stock', the item is available for immediate purchase.",
    "Do you offer gift wrapping?": "Yes, gift wrapping is available for an additional fee of 50 BDT per item.",
    "Can I pre-order items that are out of stock?": "Yes, select items can be pre-ordered. Look for the 'Pre-Order' button on product pages.",

    # Loyalty Program & Promotions
    "Do you have a loyalty program?": "Yes, our loyalty program awards you 1 point for every 100 BDT spent. 100 points can be redeemed for a 200 BDT discount.",
    "How do I join your loyalty program?": "You're automatically enrolled in our loyalty program when you create an account with us.",
    "Do you have any ongoing promotions?": "We regularly run promotions throughout the year. Check our 'Promotions' page or subscribe to our newsletter to stay updated.",
    "Do you offer birthday discounts?": "Yes, we offer a 10% discount on purchases made during your birthday month. Ensure your birth date is correctly entered in your profile.",
    "How do I use my loyalty points?": "You can use your loyalty points during checkout. Simply select 'Use Points' and choose how many points you want to redeem.",

    # Technical Support
    "The website is not working properly. What should I do?": "Please try clearing your browser cache and cookies. If the issue persists, try using a different browser or contact our technical support team.",
    "How do I use the mobile app?": "Download our app from the App Store or Google Play Store, create an account or log in, and you can start shopping immediately.",
    "My payment failed. What should I do?": "Please check your card details and available balance. If the issue persists, try using a different payment method or contact your bank.",

    # Others
    "Do you have a blog?": "Yes, we have a blog featuring product reviews, style guides, and industry news. You can access it from the footer of our website.",
    "Do you offer corporate discounts?": "Yes, we offer special discounts for corporate bulk orders. Please email corporate@example.com for more information.",
    "How can I become a supplier or vendor?": "Please send your company details and product catalog to vendors@example.com for consideration.",
    "Do you have a mobile app?": "Yes, our mobile app is available for download on both iOS and Android platforms.",
}

## Step-4: Initialize the chatbot with our FAQ data

In [5]:
chatbot = FAQChatbot(faq_data)

## Step-5: Demo function to test multiple queries

In [6]:
def demo_chatbot():
    test_queries = [
        # Basic tests matching original examples
        "Do you offer delivery?",
        "Where are you located?",
        "Can I return a product?",
        "Do you sell mobile phones?",

        # Semantic matching tests
        "How can I get my package delivered?",
        "What's your store address?",
        "I want to send back an item",
        "Do you have a store in New York?",

        # Tests for new FAQ data
        "Is my data safe with you?",
        "Can I pay in installments?",
        "My order hasn't arrived yet, what should I do?",
        "Do you have any sales going on?",
        "How do I get points in your loyalty program?",
        "I forgot my password",
        "The website is slow",
        "Do you deliver outside Dhaka?",
        "Can I exchange instead of return?",
        "I received a defective product"
    ]

    print("FAQ Chatbot Demo:\n")
    for query in test_queries:
        answer = get_faq_answer(query)
        print(f"Q: {query}")
        print(f"A: {answer}")
        print("-" * 50)
demo_chatbot()

FAQ Chatbot Demo:

Q: Do you offer delivery?
A: Yes, we provide delivery service for all orders above 500 BDT.
--------------------------------------------------
Q: Where are you located?
A: We are located at 123 Main Road, Dhaka.
--------------------------------------------------
Q: Can I return a product?
A: You can return a product within 7 days of delivery.
--------------------------------------------------
Q: Do you sell mobile phones?
A: Yes, our mobile app is available for download on both iOS and Android platforms.
--------------------------------------------------
Q: How can I get my package delivered?
A: Sorry, I couldn't find a suitable answer to your question.
--------------------------------------------------
Q: What's your store address?
A: Our physical store is open from 10 AM to 8 PM, seven days a week.
--------------------------------------------------
Q: I want to send back an item
A: Yes, you can exchange items within 10 days of delivery. The new item must be of equa

## Examples from the problem statement

In [7]:
print("Example 1:Do you offer delivery?")
print(get_faq_answer("Do you offer delivery?"))

print("\nExample 2:Where are you located?")
print(get_faq_answer("Where are you located?"))

print("\nExample 3:Can I return a product?")
print(get_faq_answer("Can I return a product?"))

print("\nExample 4:Do you sell mobile phones?")
print(get_faq_answer("Do you sell mobile phones?"))

Example 1:Do you offer delivery?
Yes, we provide delivery service for all orders above 500 BDT.

Example 2:Where are you located?
We are located at 123 Main Road, Dhaka.

Example 3:Can I return a product?
You can return a product within 7 days of delivery.

Example 4:Do you sell mobile phones?
Yes, our mobile app is available for download on both iOS and Android platforms.


## Additional tests to demonstrate the improved chatbot with expanded FAQ data

In [8]:
print("\nExpanded FAQ Testing:")
print("Query: How do I reset my password?")
print("Answer:", get_faq_answer("How do I reset my password?"))

print("\nQuery: Is my credit card information safe?")
print("Answer:", get_faq_answer("Is my credit card information safe?"))

print("\nQuery: Can I pay in EMI?")
print("Answer:", get_faq_answer("Can I pay in EMI?"))

print("\nQuery: My product is broken, what should I do?")
print("Answer:", get_faq_answer("My product is broken, what should I do?"))

print("\nQuery: Do you have discounts on my birthday?")
print("Answer:", get_faq_answer("Do you have discounts on my birthday?"))


Expanded FAQ Testing:
Query: How do I reset my password?
Answer: Click on the 'Forgot Password' link on the login page and follow the instructions sent to your registered email address.

Query: Is my credit card information safe?
Answer: We use industry-standard SSL encryption to protect your payment information. We never store your full credit card details on our servers.

Query: Can I pay in EMI?
Answer: Sorry, I couldn't find a suitable answer to your question.

Query: My product is broken, what should I do?
Answer: You can return a product within 7 days of delivery.

Query: Do you have discounts on my birthday?
Answer: Yes, we offer a 10% discount on purchases made during your birthday month. Ensure your birth date is correctly entered in your profile.
